Get Wikipedia's n-grams base on the dump data

# Download dump

In [6]:
import os
The_path=r"C:/Users/Reza/Desktop/core"
Dump_URL =u'https://dumps.wikimedia.org/kuwiki/latest/kuwiki-latest-pages-articles.xml.bz2'
# download last dump
os.system('wget '+Dump_URL +" "+The_path+"/kuwiki-latest-pages-articles.xml.bz2")

1

# Feature extraction 

This code extracts features from articles:
* Article text file
* Category of the article
* Internal links inside the articles
* File and images in articles


In [7]:
import pickle
import sys , re
# Load pywikibot library
sys.path.append(The_path+'/pywikibot')
sys.path.append(The_path)
import xmlreader
#import core
#from pywikibot import xmlreader
dump = xmlreader.XmlDump(The_path+r"/kuwiki-latest-pages-articles.xml.bz2")

farsi_char=r'abcçdeêfghiîjklmnopqrsştuûvwxyzABCÇDEÊFGHIÎJKLMNOPQRSŞTUÛVWXYZ'# Persian Char + ZWNJ
suported_extenstions=r'(?:[gt]iff?|png|jpe?g|web[mp]|xcf|pdf|mid|og[avg]|svg|djvu|flac|opus|wav|mp3)'

def flat_text (text):
    # clean and remove wiki sintax
    text=text.replace('\r','')
    text=re.sub(r'\{\{[^\}]+\}\}',' ',text)# remove wiki templates
    text=re.sub('\[\[(?:Kategorî|Wêne)\:.*?\]\]','',text) # remove categories and images
    text=re.sub(r'\<[^\>]+\>',' ',text) # remove commented texts
    text=re.sub(r'\n={1,}[^\n]+\={1,}',' ',text) # remove subtitles
    text=re.sub(r'\|[^\=\]\n]+\=',' ',text) # remove remained template variables
    text=re.sub(r'\[\[[^\|\]]+\|','',text) # to have correct text. I remove the piped text. eg. This is a [[foo|bar]] > This is bar
    text=re.sub(r'\.'+suported_extenstions,'',text)# remove useless . 
    text=text.replace('&ndash;',' ')# Replaced html space to not connect words
    text=text.replace('[[',' ').replace(']]','')# replaced ]] and [[ with empty to connect the connected char after them. eg. [[book]]s > books
    
    text=re.sub(r'[^'+farsi_char+'\.]+',' ',text)# removed none persian characters except . to find sentences
    text=re.sub(r'\.{1,}','.',text)# remove repeated .
    text=re.sub(r'(\. ){1,}','. ',text)# remove repeated .
    
    text=re.sub(r'\s+',' ',text)# Replace \n and space with one space
    text=text.replace(' . ','. ')
    return text

category_dict={}
links_dict={}
image_dict={}
title_dict={}
text_dict={}

counter=0
for entry in dump.parse():
    if entry.ns =='0' and not entry.isredirect:
        counter+=1
        text_dict[entry.id]=flat_text(entry.text)
        title_dict[entry.id]=entry.title
        #image_dict[entry.id]=get_images(entry.text)
        #links_dict[entry.id]=get_links(entry.text)
        #category_dict[entry.id]=get_categories(entry.text)
        if counter % 50000==0:
            print(counter)


In [ ]:
import stanfordnlp
config = {
    #'processors': 'tokenize,mwt,pos,lemma,depparse', # Comma-separated list of processors to use
    'processors': 'tokenize,pos',
    #'lang': 'fa', # Language code for the language to build the Pipeline in
    'lang': 'kmr', # Language code for the language to build the Pipeline in
}
pipeline = stanfordnlp.Pipeline(**config)#lang='fa', use_gpu=True)

In [ ]:
with open(r"C:\Users\Reza\Desktop\ku\ku_text_dict.txt", "r", encoding="utf-8") as f:
    text=f.read()
len (text)
for i in range(0,2000000,50000):
    try:
        doc = pipeline(text[i:50000+i])
        with open(r"C:\Users\Reza\Desktop\کردی\ku_tokens_2.txt","a", encoding="utf-8") as f:
            f.write(u'\n'+doc.conll_file.conll_as_string())
    except:
        continue
